In [1]:
import os
from os.path import dirname

root_dir = dirname(os.getcwd())
os.chdir(root_dir)

In [2]:
import pickle

with open('data/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from src.nn.dataloader import get_dataloader
from src.nn.model import NeuralNetwork

In [4]:
train_dl = get_dataloader(train_data)
test_dl = get_dataloader(test_data)

In [5]:
num_epoch = 100

net = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

for epoch in range(num_epoch):
    pred_label = []
    true_label = []
    net.train()
    running_loss = 0.0

    for data in train_dl:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        pred_label.extend(labels.cpu().numpy())
        true_label.extend(preds.cpu().numpy())

    epoch_loss = running_loss / len(train_dl)
    acc = accuracy_score(pred_label, true_label)
    prec = precision_score(pred_label, true_label, average='macro', zero_division=0)
    rec = recall_score(pred_label, true_label, average='macro', zero_division=0)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}')

Epoch 10, Loss: 0.8982, Acc: 0.9042, Prec: 0.9078, Rec: 0.9042
Epoch 20, Loss: 0.7826, Acc: 0.9792, Prec: 0.9798, Rec: 0.9792
Epoch 30, Loss: 0.7566, Acc: 0.9958, Prec: 0.9959, Rec: 0.9958
Epoch 40, Loss: 0.7513, Acc: 0.9958, Prec: 0.9959, Rec: 0.9958
Epoch 50, Loss: 0.7495, Acc: 0.9958, Prec: 0.9959, Rec: 0.9958
Epoch 60, Loss: 0.7480, Acc: 0.9958, Prec: 0.9959, Rec: 0.9958
Epoch 70, Loss: 0.7451, Acc: 1.0000, Prec: 1.0000, Rec: 1.0000
Epoch 80, Loss: 0.7446, Acc: 1.0000, Prec: 1.0000, Rec: 1.0000
Epoch 90, Loss: 0.7443, Acc: 1.0000, Prec: 1.0000, Rec: 1.0000
Epoch 100, Loss: 0.7442, Acc: 1.0000, Prec: 1.0000, Rec: 1.0000


In [7]:
pred_label = []
true_label = []

for data in test_dl:
    net.eval()
    inputs, labels = data
    outputs = net(inputs)
    _, preds = torch.max(outputs, 1)
    pred_label.extend(preds.cpu().numpy())
    true_label.extend(labels.cpu().numpy())

acc = accuracy_score(pred_label, true_label)
prec = precision_score(pred_label, true_label, average='macro', zero_division=0)
rec = recall_score(pred_label, true_label, average='macro', zero_division=0)
print(f'Test Set - Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}')

Test Set - Acc: 0.7500, Prec: 0.7500, Rec: 0.7527
